For the purpose of creating a playground-like teaching and learning environment, all the code used here is stored in a ipython or jupyter notebook. import_ipynb is used to import classes and functions from other notebooks. Since these imports will run all the cells in the notebook you are importing from, once you have finished playing with a certain module or cell in a notebook, comment out the cells you would rather not have executed when the notebook is imported. 

In [ ]:
import math, copy
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import import_ipynb
from notebooks.Data_mover import csv2datatools, Options, num_batches
from notebooks.Encoder import Encoder 
from notebooks.Decoder import Decoder 
from notebooks.Trainer import trainer, CosineWithRestarts
from notebooks.Talk import talk_to_model

import nltk
# for synonym finding, nltk needs to download a folder called corpora to your computer, 
# change corpora_path to your prefered folder, '../' will put it in the parent folder
corpora_path = '../'
nltk.download('wordnet', corpora_path) 
from nltk.corpus import wordnet

%load_ext autoreload
%autoreload 2

In [2]:
opt = Options(batchsize = 4)
data_iter, infield, outfield, opt = csv2datatools('saved/translation_pairs.csv','en', opt)

In [3]:
class Transformer(nn.Module):
    def __init__(self, in_vocab_size, out_vocab_size, emb_dim, n_layers, heads, dropout):
        super().__init__()
        self.encoder = Encoder(in_vocab_size, emb_dim, n_layers, heads, dropout)
        self.decoder = Decoder(out_vocab_size, emb_dim, n_layers, heads, dropout)
        self.out = nn.Linear(emb_dim, out_vocab_size)
    def forward(self, src_seq, trg_seq, src_mask, trg_mask):
        e_output = self.encoder(src_seq, src_mask)
        d_output = self.decoder(trg_seq, e_output, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [4]:
emb_dim, n_layers, heads, dropout = 64, 2, 8, 0.1 
opt.save_path = 'saved/weights/model_weights'
model = Transformer(len(infield.vocab), len(outfield.vocab), emb_dim, n_layers, heads, dropout)

if opt.device != -1:
    model = model.cuda()
    
model.load_state_dict(torch.load(opt.save_path))

<All keys matched successfully>

In [5]:
opt.lr = 0.001 #0.0001
opt.epochs = 20 
optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)
scheduler = CosineWithRestarts(optimizer, T_max=num_batches(data_iter))

In [ ]:
model = trainer(model, data_iter, opt, optimizer, scheduler)

In [5]:
sentence = "who are you?" 
opt.k = 1
opt.max_len = 10
sentence = talk_to_model(sentence, model, opt, infield, outfield)
print('Chloe > '+ sentence + '\n')

Chloe > i am chloe



In [ ]:
# when you want to turn off this cell, click Kernel->Interrupt 
while True:
    sentence = input("You > ")
    sentence = talk_to_model(sentence, model, opt, infield, outfield)
    print('Chloe > '+ sentence + '\n')

You > hi
Chloe > hello

You > who are you?
Chloe > i am chloe

